In [2]:
from sdv.metadata import Metadata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [7]:
df_m_0_df_0 ="min_0_informative_cluster_0"
df_m_1_df_0 ="min_1_informative_cluster_0"
df_m_0_df_1 ="min_0_informative_cluster_1"
df_m_1_df_1 ="min_1_informative_cluster_1"
df_m_2_df_1 ="min_2_informative_cluster_1"
df_m_2_df_0 ="min_2_informative_cluster_0"


df_m_0_df_m_1_df_0_df_1 ="min_0_min_1_informative_cluster_0_informative_cluster_1"
df_m_0_df_m_2_df_0_df_1 ="min_0_min_2_informative_cluster_0_informative_cluster_1"
df_m_1_df_m_2_df_0_df_1 ="min_1_min_2_informative_cluster_0_informative_cluster_1"


selected_file = df_m_0_df_0
files = [df_m_0_df_0, df_m_0_df_1,df_m_1_df_1, df_m_1_df_0, df_m_2_df_0,df_m_2_df_1,df_m_0_df_m_1_df_0_df_1,df_m_0_df_m_2_df_0_df_1,df_m_1_df_m_2_df_0_df_1 ]
df= pd.read_csv("../dataset/" + selected_file + ".csv")
df_fake = pd.read_csv("../dataset/fake_" + selected_file + ".csv")


In [8]:
df= df.drop(columns=["Unnamed: 0"])
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,181.00,181.00,0.0,21182.00,0.00
1,1,2806.00,2806.00,0.0,26202.00,0.00
2,1,20128.00,20128.00,0.0,6268.00,12145.85
3,1,25071.46,25071.46,0.0,9083.76,34155.22
4,1,235238.66,235238.66,0.0,0.00,235238.66


In [9]:
df_fake.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,287063.71,75273.99,5617.94,322547.24,668726.34
1,1,395595.06,1018223.74,1918819.84,6529.50,436731.36
2,1,2712398.88,3274272.38,26028.67,108318.48,1683404.60
3,1,134198.61,102083.72,7470.55,2609554.08,1577920.23
4,1,1640298.36,1771092.55,14375.07,89408.37,9347357.47


This metric measures the correlation between a pair of numerical columns and computes the similarity between the real and synthetic data -- aka it compares the trends of 2D distributions.

Let's try amount and newbalanceDest
If all clusters results was high -> i have totally noisy random distribution

In [20]:
from sdmetrics.column_pairs import CorrelationSimilarity
correlation_similarities = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if "Unnamed: 0" in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    correlation_similarity = CorrelationSimilarity.compute(
    real_data=df_org[["amount","newbalanceDest"]],
    synthetic_data=df_fake[["amount","newbalanceDest"]],
    coefficient='Spearman'
    )
    print(correlation_similarity)
    correlation_similarities.append(correlation_similarity)


min_0_informative_cluster_0
0.8146360385316167
min_0_informative_cluster_1
0.948328853115306
min_1_informative_cluster_1
0.955977572111264
min_1_informative_cluster_0
0.8447380375163316
min_2_informative_cluster_0
0.9657846559884142
min_2_informative_cluster_1
0.9439824471270863
min_0_min_1_informative_cluster_0_informative_cluster_1
0.9999496787943054
min_0_min_2_informative_cluster_0_informative_cluster_1
0.9079373887842288
min_1_min_2_informative_cluster_0_informative_cluster_1
0.9663731874677317


KSComplement, This metric computes the similarity of a real column vs. a synthetic column in terms of the column shapes -- aka the marginal distribution or 1D histogram of the column.

marginal distribution or 1D histogram

In [12]:
from sdmetrics.single_column import KSComplement
correlation_similarities = []
column_scores = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if "Unnamed: 0" in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    for column in df_org.columns:
        if column in ["type"]:
            continue
        ks_comp = KSComplement.compute(
        real_data=df_org[column],
        synthetic_data=df_fake[column],
        )
        # print(column)
        # print(ks_comp)
        column_scores.append(ks_comp)

    avg = sum(column_scores)/len(column_scores)
    print(avg)
    correlation_similarities.append(avg)
    column_scores = []


min_0_informative_cluster_0
0.8240524264966348
min_0_informative_cluster_1
0.7474006116207951
min_1_informative_cluster_1
0.7785604606525912
min_1_informative_cluster_0
0.7233131024634059
min_2_informative_cluster_0
0.8212953876349364
min_2_informative_cluster_1
0.7498833138856476
min_0_min_1_informative_cluster_0_informative_cluster_1
0.7514751649446035
min_0_min_2_informative_cluster_0_informative_cluster_1
0.7703407454807231
min_1_min_2_informative_cluster_0_informative_cluster_1
0.7238882645689517


StatisticSimilarity: This metric measures the similarity between a real column and a synthetic column by comparing a summary statistic. Supported summary statistics are: mean, median and standard deviation.


In [19]:
import math
from sdmetrics.single_column import StatisticSimilarity
correlation_similarities = []
column_scores = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if "Unnamed: 0" in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    for column in df_org.columns:
        if column in ["type"]:
            continue
        ks_comp = StatisticSimilarity.compute(
        real_data=df_org[column],
        synthetic_data=df_fake[column],
        statistic="std"
        )
        if math.isnan(ks_comp):
            continue
        # print(column)
        # print(ks_comp)
        column_scores.append(ks_comp)

    avg = sum(column_scores)/len(column_scores)
    print(avg)
    correlation_similarities.append(avg)
    column_scores = []


min_0_informative_cluster_0
0.9913584924703223
min_0_informative_cluster_1
0.9892459611000692
min_1_informative_cluster_1
0.9902567033509811
min_1_informative_cluster_0
0.9786089131459846
min_2_informative_cluster_0
0.9852325596967788
min_2_informative_cluster_1
0.9875992281449637
min_0_min_1_informative_cluster_0_informative_cluster_1
0.9896176647257814
min_0_min_2_informative_cluster_0_informative_cluster_1
0.9889704687903145
min_1_min_2_informative_cluster_0_informative_cluster_1
0.9892582002552782


TVComplement:
This metric computes the similarity of a real column vs. a synthetic column in terms of the column shapes -- aka the marginal distribution or 1D histogram of the column.
Only for categorical data

In [15]:
from sdmetrics.single_column import TVComplement
correlation_similarities = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if "Unnamed: 0" in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    ks_comp = TVComplement.compute(
    real_data=df_org["type"],
    synthetic_data=df_fake["type"],
    )

    print(ks_comp)


min_0_informative_cluster_0
0.9888416578108395
min_0_informative_cluster_1
0.9003249235474007
min_1_informative_cluster_1
0.9741842610364683
min_1_informative_cluster_0
0.9700107104605498
min_2_informative_cluster_0
0.9195289499509323
min_2_informative_cluster_1
0.9600350058343057
min_0_min_1_informative_cluster_0_informative_cluster_1
0.9251836175774929
min_0_min_2_informative_cluster_0_informative_cluster_1
0.9330507118860982
min_1_min_2_informative_cluster_0_informative_cluster_1
0.9723872210627709


Privacy metrics

The DCRBaselineProtection metric measures the distance between your synthetic data and real data to measure how private it is. For a fair measurement, it compares the distance against randomly generated data, which would provide the best possible privacy protection.

DCR = Distance to Closest Record


In [16]:
from sdmetrics.single_table import DCRBaselineProtection
from sdv.metadata import Metadata

correlation_similarities = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if "Unnamed: 0" in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    ks_comp = DCRBaselineProtection.compute_breakdown(
    real_data=df_org,
    synthetic_data=df_fake,
    metadata= Metadata.detect_from_dataframe(df_org).to_dict().get("tables").get("table")
    )

    print(ks_comp['score'])


min_0_informative_cluster_0
0.03175558368342471
min_0_informative_cluster_1
0.011997589031625077
min_1_informative_cluster_1
0.014894614462013804
min_1_informative_cluster_0
0.01967817193059218
min_2_informative_cluster_0
0.019654511316558595
min_2_informative_cluster_1
0.0006817255309615392
min_0_min_1_informative_cluster_0_informative_cluster_1
0.005704315080208666
min_0_min_2_informative_cluster_0_informative_cluster_1
0.0017217224917579442
min_1_min_2_informative_cluster_0_informative_cluster_1
0.0014179871363931105


(best) 1.0: The synthetic data has the highest possible privacy protection. Replacing the synthetic data entirely with random data would not improve the privacy.

(worst) 0.0: The synthetic has the worst possible privacy protection. Compared to random data, the synthetic data is much closer to the real data.

The DisclosureProtection metric measures the risk associated with disclosing (aka broadly sharing) the synthetic data. It's a useful measurement if you want to know whether synthetic data is leaking patterns that pertain to sensitive information.

In [17]:
from sdmetrics.single_table import DisclosureProtection
from sdv.metadata import Metadata

correlation_similarities = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if "Unnamed: 0" in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    ks_comp = DisclosureProtection.compute(
    real_data=df_org,
    synthetic_data=df_fake,
    known_column_names=['newbalanceOrig','newbalanceDest','oldbalanceDest'],
    sensitive_column_names=['amount'],
    )

    print(ks_comp)


min_0_informative_cluster_0
1
min_0_informative_cluster_1


I:\ETVOS\2nd semester\DS Lab 1\DSLAB-Project\venv1\Lib\site-packages\sdmetrics\single_table\privacy\disclosure_protection.py:214: UserWarning: Data exceeds 10000 rows, perfomance may be slow. Consider using the `DisclosureProtectionEstimate` for faster computation.
  warnings.warn(


1
min_1_informative_cluster_1


I:\ETVOS\2nd semester\DS Lab 1\DSLAB-Project\venv1\Lib\site-packages\sdmetrics\single_table\privacy\disclosure_protection.py:214: UserWarning: Data exceeds 10000 rows, perfomance may be slow. Consider using the `DisclosureProtectionEstimate` for faster computation.
  warnings.warn(


1
min_1_informative_cluster_0
1
min_2_informative_cluster_0
1
min_2_informative_cluster_1
1
min_0_min_1_informative_cluster_0_informative_cluster_1


...

1
min_0_min_2_informative_cluster_0_informative_cluster_1


I:\ETVOS\2nd semester\DS Lab 1\DSLAB-Project\venv1\Lib\site-packages\sdmetrics\single_table\privacy\disclosure_protection.py:214: UserWarning: Data exceeds 10000 rows, perfomance may be slow. Consider using the `DisclosureProtectionEstimate` for faster computation.
  warnings.warn(


1
min_1_min_2_informative_cluster_0_informative_cluster_1


I:\ETVOS\2nd semester\DS Lab 1\DSLAB-Project\venv1\Lib\site-packages\sdmetrics\single_table\privacy\disclosure_protection.py:214: UserWarning: Data exceeds 10000 rows, perfomance may be slow. Consider using the `DisclosureProtectionEstimate` for faster computation.
  warnings.warn(


1


The whole quality report

The Single Table Quality Report evaluates how well your synthetic data captures mathematical properties in your data.

AVG between Evaluating Column Shapes and Evaluating Column Pair Trends for each coloumn


In [18]:
from sdmetrics.reports.single_table import QualityReport

report = QualityReport()

for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if "Unnamed: 0" in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    report.generate(df_org, df_fake, Metadata.detect_from_dataframe(df_org).to_dict().get("tables").get("table"),verbose=False)
    print(report.get_score())
    # print(report.get_details(property_name='Column Shapes'))


min_0_informative_cluster_0
0.8667302094093139
min_0_informative_cluster_1
0.8316788393168555
min_1_informative_cluster_1
0.8766075840507723
min_1_informative_cluster_0
0.834356579304972
min_2_informative_cluster_0
0.8329007767091074
min_2_informative_cluster_1
0.8181600243620659
min_0_min_1_informative_cluster_0_informative_cluster_1
0.8401545105219063
min_0_min_2_informative_cluster_0_informative_cluster_1
0.8488720410133966
min_1_min_2_informative_cluster_0_informative_cluster_1
0.8232079053202237
